### Adding new variables to NetCDF

We create the weighted average SM estimates directly using xarray

In [8]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import os
import numpy as np

path2data = "/Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/"
path2out = "/Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/processed/"
path2Wang = "/Users/tejasvi/Dropbox/Database/Hydrology/Wang_2021_Soil_Moisture/processed/"

# Check if the folder exists
if not os.path.exists(path2out):
    # If the folder doesn't exist, create it
    os.makedirs(path2out)
    print(f"Folder created at: {path2out}")
else:
    # If the folder exists, do nothing
    print(f"Folder already exists at: {path2out}")

Folder already exists at: /Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/processed/


In [9]:
# Open the NetCDF file
ds = xr.open_dataset(path2data + "output_remapped.nc")
print(ds)

<xarray.Dataset> Size: 2GB
Dimensions:       (time: 552, lon: 720, lat: 360, depth: 1, bnds: 2,
                   depth_2: 1, depth_3: 1, depth_4: 1)
Coordinates:
  * time          (time) datetime64[ns] 4kB 1979-01-01 1979-02-01 ... 2024-12-01
  * lon           (lon) float64 6kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat           (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * depth         (depth) float64 8B 0.0
  * depth_2       (depth_2) float64 8B 7.0
  * depth_3       (depth_3) float64 8B 28.0
  * depth_4       (depth_4) float64 8B 100.0
Dimensions without coordinates: bnds
Data variables:
    depth_bnds    (depth, bnds) float64 16B ...
    depth_2_bnds  (depth_2, bnds) float64 16B ...
    depth_3_bnds  (depth_3, bnds) float64 16B ...
    depth_4_bnds  (depth_4, bnds) float64 16B ...
    swvl1         (time, depth, lat, lon) float32 572MB ...
    swvl2         (time, depth_2, lat, lon) float32 572MB ...
    swvl3         (time, depth_3, lat, lon) float32

In [10]:
# Remove singleton depth dimensions (drop them completely)
swvl1 = ds['swvl1'].squeeze(drop=True)  # shape: (time, lat, lon)
swvl2 = ds['swvl2'].squeeze(drop=True)
swvl3 = ds['swvl3'].squeeze(drop=True)
swvl4 = ds['swvl4'].squeeze(drop=True)

# Now safely stack along a new 'depth' dimension
stacked = xr.concat([swvl1, swvl2, swvl3, swvl4], dim='depth')
stacked.coords['depth'] = [0, 7, 28, 100]  # optional, for metadata

# Define and normalize weights
weights = xr.DataArray([7, 21, 72, 189], dims='depth')
weights = weights / weights.sum()

# Compute weighted average
swvl_0_289 = (stacked * weights).sum(dim='depth')

# Add to original dataset
ds['swvl_0_289'] = swvl_0_289

print(ds)

<xarray.Dataset> Size: 3GB
Dimensions:       (time: 552, lon: 720, lat: 360, depth: 1, bnds: 2,
                   depth_2: 1, depth_3: 1, depth_4: 1)
Coordinates:
  * time          (time) datetime64[ns] 4kB 1979-01-01 1979-02-01 ... 2024-12-01
  * lon           (lon) float64 6kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat           (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * depth         (depth) float64 8B 0.0
  * depth_2       (depth_2) float64 8B 7.0
  * depth_3       (depth_3) float64 8B 28.0
  * depth_4       (depth_4) float64 8B 100.0
Dimensions without coordinates: bnds
Data variables:
    depth_bnds    (depth, bnds) float64 16B ...
    depth_2_bnds  (depth_2, bnds) float64 16B ...
    depth_3_bnds  (depth_3, bnds) float64 16B ...
    depth_4_bnds  (depth_4, bnds) float64 16B ...
    swvl1         (time, depth, lat, lon) float32 572MB ...
    swvl2         (time, depth_2, lat, lon) float32 572MB ...
    swvl3         (time, depth_3, lat, lon) float32

In [11]:
# Now safely stack along a new 'depth' dimension
stacked = xr.concat([swvl1, swvl2, swvl3], dim='depth')
stacked.coords['depth'] = [0, 7, 28]  # optional, for metadata

# Define and normalize weights
weights = xr.DataArray([7, 21, 72], dims='depth')
weights = weights / weights.sum()

# Compute weighted average
swvl_0_100 = (stacked * weights).sum(dim='depth')

# Add to original dataset
ds['swvl_0_100'] = swvl_0_100

print(ds)

<xarray.Dataset> Size: 5GB
Dimensions:       (time: 552, lon: 720, lat: 360, depth: 1, bnds: 2,
                   depth_2: 1, depth_3: 1, depth_4: 1)
Coordinates:
  * time          (time) datetime64[ns] 4kB 1979-01-01 1979-02-01 ... 2024-12-01
  * lon           (lon) float64 6kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat           (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * depth         (depth) float64 8B 0.0
  * depth_2       (depth_2) float64 8B 7.0
  * depth_3       (depth_3) float64 8B 28.0
  * depth_4       (depth_4) float64 8B 100.0
Dimensions without coordinates: bnds
Data variables:
    depth_bnds    (depth, bnds) float64 16B ...
    depth_2_bnds  (depth_2, bnds) float64 16B ...
    depth_3_bnds  (depth_3, bnds) float64 16B ...
    depth_4_bnds  (depth_4, bnds) float64 16B ...
    swvl1         (time, depth, lat, lon) float32 572MB ...
    swvl2         (time, depth_2, lat, lon) float32 572MB ...
    swvl3         (time, depth_3, lat, lon) float32

Now, convert to yearly using xarray again

In [14]:
# Select only the variables you're interested in
ds_sel = ds[["swvl_0_289", "swvl_0_100"]]

# Group by year and compute the mean
ds_yearly = ds_sel.groupby("time.year").mean("time")

# Optionally rename 'year' dimension back to 'time' with datetime index
# If you want real datetime timestamps:
ds_yearly = ds_yearly.rename({'year': 'time'})
ds_yearly['time'] = xr.cftime_range(start=str(ds_yearly.time.values[0]), periods=len(ds_yearly.time), freq='YS')

# Convert longitudes from [0, 360] to [-180, 180] (To match Wang et al./fishnet)
ds_yearly['lon'] = ((ds_yearly['lon'] + 180) % 360) - 180

# Sort the dataset along the new lon axis (important!)
ds_yearly = ds_yearly.sortby('lon')

print(ds_yearly)

# Save to NetCDF
#ds_yearly.to_netcdf(path2out + "swvl_yearly_avg.nc")

<xarray.Dataset> Size: 191MB
Dimensions:     (time: 46, lat: 360, lon: 720)
Coordinates:
  * lon         (lon) float64 6kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat         (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time        (time) object 368B 1979-01-01 00:00:00 ... 2024-01-01 00:00:00
Data variables:
    swvl_0_289  (time, lat, lon) float64 95MB 0.1659 0.1656 0.1654 ... 0.0 0.0
    swvl_0_100  (time, lat, lon) float64 95MB 0.2055 0.2048 0.2041 ... 0.0 0.0
Attributes:
    CDI:          Climate Data Interface version 2.1.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Fri Apr 04 21:15:59 2025: cdo -O -s -f nc -copy /Users/teja...
    CDO:          Climate Data Operators version 2.1.1 (https://mpimet.mpg.de...


/var/folders/5n/knp134qs6wqgkbfwmwcrs8vw0000gn/T/ipykernel_8014/411229372.py:10: DeprecationWarning: cftime_range() is deprecated, please use xarray.date_range(..., use_cftime=True) instead.
  ds_yearly['time'] = xr.cftime_range(start=str(ds_yearly.time.values[0]), periods=len(ds_yearly.time), freq='YS')


### Save the yearly as CSV

In [15]:
# Select specific variables to include (e.g., swvl1, swvl2)
selected_vars = ["swvl_0_289", "swvl_0_100"]

# Convert selected variables to a long-format DataFrame
df = ds_yearly[selected_vars].to_dataframe().reset_index()

# Convert to Dask DataFrame
df = dd.from_pandas(df, npartitions=4)

# Preview the DataFrame
print(df.head())
print(df.shape)


                  time    lat     lon  swvl_0_289  swvl_0_100
0  1979-01-01 00:00:00 -89.75 -179.75    0.165884    0.205519
1  1979-01-01 00:00:00 -89.75 -179.25    0.165637    0.204805
2  1979-01-01 00:00:00 -89.75 -178.75    0.165387    0.204084
3  1979-01-01 00:00:00 -89.75 -178.25    0.165140    0.203368
4  1979-01-01 00:00:00 -89.75 -177.75    0.164892    0.202652
(<dask_expr.expr.Scalar: expr=df.size() // 5, dtype=int64>, 5)


In [ ]:
# Ensure the date_column is of string type
df['time'] = df['time'].astype(str)

# Extract 'yearmon' (first 7 characters: 'YYYY-MM')
df['yearmon'] = df['time'].str[:7]

# Extract 'year' (first 4 characters: 'YYYY')
df['year'] = df['time'].str[:4]

# Extract 'month' (characters at position 6-7: 'MM')
#df['month'] = df['time'].str[5:7]

# Select and re-order the columns as requested
selected_columns = [
    'lon', 'lat', 'yearmon', 'year',
    'swvl_0_100', 'swvl_0_289'
]

# Re-order the DataFrame based on the selected columns
df = df[selected_columns]

# Filter latitude to between -54.75 and 78.75
df = df[(df['lat'] >= -54.75) & (df['lat'] <= 78.75)]

print(df.head())#

          lon    lat  yearmon  year  swvl_0_100  swvl_0_289
50400 -179.75 -54.75  1979-01  1979         0.0         0.0
50401 -179.25 -54.75  1979-01  1979         0.0         0.0
50402 -178.75 -54.75  1979-01  1979         0.0         0.0
50403 -178.25 -54.75  1979-01  1979         0.0         0.0
50404 -177.75 -54.75  1979-01  1979         0.0         0.0


In [ ]:

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the full file path with the correct file name
output_file = os.path.join(output_dir, "ERA5_land_sm_data_05deg_1979_2024.csv")

# Check the final output file path
print(f"Output file path: {output_file}")

# Save to CSV
#df_pandas = df.compute()  # Convert to Pandas DataFrame
#df_pandas.to_csv(output_file, index=False)

print(f"File saved at: {output_file}")

Output file path: /Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/processed/ERA5_land_sm_data_05deg_1979_2024.csv
File saved at: /Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/processed/ERA5_land_sm_data_05deg_1979_2024.csv


### Load the Wang Data

In [ ]:
path2Wang = "/Users/tejasvi/Dropbox/Database/Hydrology/Wang_2021_Soil_Moisture/processed/"
# Load the CSV into a DataFrame
df_wang = pd.read_csv(path2Wang + "sm_data_monthly_1970_2016.csv")

# Convert to Dask DataFrame
df_wang = dd.from_pandas(df_wang, npartitions=4)

# Preview the DataFrame
print(df_wang.shape)
print(df_wang.head())

(<dask_expr.expr.Scalar: expr=df.size() // 7, dtype=int64>, 7)
      lon    lat  yearmon  year  month  sm_0_100  sm_0_100_rescaled
0 -179.75  65.75  1970-01  1970      1  0.269013           0.291638
1 -179.75  65.75  1970-02  1970      2  0.268694           0.282631
2 -179.75  65.75  1970-03  1970      3  0.268707           0.283012
3 -179.75  65.75  1970-04  1970      4  0.268765           0.284649
4 -179.75  65.75  1970-05  1970      5  0.270480           0.333092
<bound method FrameBase.min of Dask Series Structure:
npartitions=4
0           float64
7953387         ...
15906774        ...
23860161        ...
31813547        ...
Dask Name: getitem, 2 expressions
Expr=df['lon']> <bound method FrameBase.max of Dask Series Structure:
npartitions=4
0           float64
7953387         ...
15906774        ...
23860161        ...
31813547        ...
Dask Name: getitem, 2 expressions
Expr=df['lon']>


In [22]:
# Assume your Dask DataFrame is called df
# Step 1: Create a proper time column
df_wang['time'] = dd.to_datetime(df_wang['year'].astype(str) + '-' + df_wang['month'].astype(str).str.zfill(2) + '-01')

# Step 2: Compute to Pandas
pdf = df_wang.compute()

# Step 3: Convert to xarray using from_dataframe
ds_wang = xr.Dataset.from_dataframe(pdf.set_index(['time', 'lat', 'lon']))

print(ds_wang)

<xarray.Dataset> Size: 4GB
Dimensions:            (time: 564, lat: 268, lon: 679)
Coordinates:
  * time               (time) datetime64[ns] 5kB 1970-01-01 ... 2016-12-01
  * lat                (lat) float64 2kB -54.75 -54.25 -53.75 ... 78.25 78.75
  * lon                (lon) float64 5kB -179.8 -179.2 -178.8 ... 179.2 179.8
Data variables:
    yearmon            (time, lat, lon) object 821MB nan nan nan ... nan nan nan
    year               (time, lat, lon) float64 821MB nan nan nan ... nan nan
    month              (time, lat, lon) float64 821MB nan nan nan ... nan nan
    sm_0_100           (time, lat, lon) float64 821MB nan nan nan ... nan nan
    sm_0_100_rescaled  (time, lat, lon) float64 821MB nan nan nan ... nan nan


In [27]:
# Assuming your xarray dataset is called ds

# Select only the variables you're interested in
ds_sel = ds_wang[["sm_0_100"]]

# Step 1: Group by year and calculate the mean
ds_yearly = ds_sel.groupby("time.year").mean("time")

# Step 2: Optionally, you can rename the 'year' coordinate back to 'time' with datetime values
ds_yearly = ds_yearly.rename({'year': 'time'})

# Create the corresponding datetime values for each year
ds_yearly['time'] = xr.cftime_range(start=str(ds_yearly.time.values[0]), periods=len(ds_yearly.time), freq='YS')

# Step 3: Export the result to NetCDF
ds_yearly.to_netcdf(path2Wang + "yearly_avg.nc")

print(ds_yearly)

<xarray.Dataset> Size: 68MB
Dimensions:   (time: 47, lat: 268, lon: 679)
Coordinates:
  * lat       (lat) float64 2kB -54.75 -54.25 -53.75 ... 77.75 78.25 78.75
  * lon       (lon) float64 5kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * time      (time) object 376B 1970-01-01 00:00:00 ... 2016-01-01 00:00:00
Data variables:
    sm_0_100  (time, lat, lon) float64 68MB nan nan nan nan ... nan nan nan nan


/var/folders/5n/knp134qs6wqgkbfwmwcrs8vw0000gn/T/ipykernel_94903/2330183468.py:13: DeprecationWarning: cftime_range() is deprecated, please use xarray.date_range(..., use_cftime=True) instead.
  ds_yearly['time'] = xr.cftime_range(start=str(ds_yearly.time.values[0]), periods=len(ds_yearly.time), freq='YS')
